In [1]:
# Ensure TensorFlow is installed
%pip install tensorflow

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

Note: you may need to restart the kernel to use updated packages.


2025-04-15 06:18:26.716719: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 06:18:26.720551: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 06:18:26.729736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744697906.753813   25377 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744697906.758171   25377 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744697906.770995   25377 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [8]:
%pip install opencv-python
%pip install imutils
import numpy as np 
from tqdm import tqdm
import cv2
import os
import imutils


Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25872 sha256=1458e49cca3b66ebc6e9ba3ae95f96c5e30493aa25e1cfb1c7b3d7e1a7f4c026
  Stored in directory: /home/codespace/.cache/pip/wheels/5b/76/96/ad0c321506837bef578cf3008df3916c23018435a355d9f6b1
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [9]:
def crop_img(img):
	"""
	Finds the extreme points on the image and crops the rectangular out of them
	"""
	gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
	gray = cv2.GaussianBlur(gray, (3, 3), 0)

	# threshold the image, then perform a series of erosions +
	# dilations to remove any small regions of noise
	thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
	thresh = cv2.erode(thresh, None, iterations=2)
	thresh = cv2.dilate(thresh, None, iterations=2)

	# find contours in thresholded image, then grab the largest one
	cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	cnts = imutils.grab_contours(cnts)
	c = max(cnts, key=cv2.contourArea)

	# find the extreme points
	extLeft = tuple(c[c[:, :, 0].argmin()][0])
	extRight = tuple(c[c[:, :, 0].argmax()][0])
	extTop = tuple(c[c[:, :, 1].argmin()][0])
	extBot = tuple(c[c[:, :, 1].argmax()][0])
	ADD_PIXELS = 0
	new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
	
	return new_img

In [14]:


training = "../Dataset/Training"
testing = "../Dataset/Testing"
training_dir = os.listdir(training)
testing_dir = os.listdir(testing)
IMG_SIZE = 256

for dir in training_dir:
	save_path = 'cleaned/Training/'+ dir
	path = os.path.join(training,dir)
	image_dir = os.listdir(path)
	for img in image_dir:
		image = cv2.imread(os.path.join(path,img))
		new_img = crop_img(image)
		new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
		if not os.path.exists(save_path):
			os.makedirs(save_path)
		cv2.imwrite(save_path+'/'+img, new_img)

for dir in testing_dir:
	save_path = 'cleaned/Testing/'+ dir
	path = os.path.join(testing,dir)
	image_dir = os.listdir(path)
	for img in image_dir:
		image = cv2.imread(os.path.join(path,img)) 
		new_img = crop_img(image)		
		new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
		if not os.path.exists(save_path):
			os.makedirs(save_path)
		cv2.imwrite(save_path+'/'+img, new_img)

FileNotFoundError: [Errno 2] No such file or directory: '../Dataset/Training'